# Making a mock observation from the EAGLE simulations

This script is an example script to lay out the individual steps of making mock observations.

The steps are as follows:

1. Load the EAGLE simulated data.  Specify the distance at which we want to observe the cosmic web i.e. redshift of central wavelength of filter and the resolution that the data should be binned to.  The minimum resolution at 50Mpc away is about 14".
2. Cut out a chunk of the simulated data that corresponds to the Dragonfly FOV.  (switched with step 3 because takes too long for the whole simulation)
3. Add the noise to the simulated data to mimic observational noises (includes sky background noise, read out noise, dark current noise, and shot noise).  This depends on the exposure time of the mock observation.
4. Plot the data to check and see what it looks like.

Ta da!

In [11]:
import numpy as np
import eagle_constants_and_units as c
import cosmo_utils as csu
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import mpl_toolkits.axes_grid1 as axgrid
from astropy import constants as const
from astropy import units as u

import os
import get_halpha_SB

%matplotlib inline

%run 'load_data.ipynb'
%run 'add_noise.ipynb'
%run 'extract_FOV.ipynb'

{'92Mpc': 0.00044639550000000005, '100Mpc': 0.00048820950000000004, '50Mpc': 0.0002397255, '200Mpc': 0.0009716159999999999, '46Mpc': 0.00022336469999999998, '500Mpc': 0.0024393600000000002}


In [13]:
machine='chinook'

def get_noisydata(resolution,distance,slicewidth,exptime,CMOS=False,ROnoise=None,x_center=47.5,y_center=12.):
    print "****** Working on mock observation for distance %s ******"%distance
    print 'loading the data...'
    data_tuple = loaddata(machine=machine,resolution=resolution,distance=distance,slicewidth=slicewidth)
    print 'extracting the FOV...'
    data_FOV, xystarts, size = extractFOV(data_tuple,resolution,distance,x_center=x_center,y_center=y_center)
    filterwidth = 3.
    if int(slicewidth) == 20:
        filterwidth=3.
    elif int(slicewidth) == 5:
        filterwidth=1.
    else:
        print "ERROR: not sure what filter width to use..."
    print 'adding noise to the data...'
    noiseadded_signal,B_sky_array,R_array = addnoise(data_FOV,resolution,exptime=exptime,filterwidth=filterwidth,\
                                                     CMOS=CMOS,R=ROnoise,debugging=True)
    
    print 'saving the noise-added data...'
    if CMOS:
        cams = 'newcam'
    else:
        cams = 'oldcam'
    if ROnoise:
        cams = cams+'_ROnoise%s'%ROnoise

    fname = 'noisy_fov_%shr_%s_%sdist_%sarcsec_%sslwd_xycenter_%s_%s.npz' \
            % (round(exptime/3600.),cams,distance,resolution,slicewidth,x_center,y_center)
    R_fname = 'noisy_fov_%shr_%s_%sdist_%sarcsec_%sslwd_xycenter_%s_%s_RARRAY.npz' \
            % (round(exptime/3600.),cams,distance,resolution,slicewidth,x_center,y_center)
    B_sky_fname = 'noisy_fov_%shr_%s_%sdist_%sarcsec_%sslwd_xycenter_%s_%s_BSKYARRAY.npz' \
            % (round(exptime/3600.),cams,distance,resolution,slicewidth,x_center,y_center)

    print "Saving mock observation as %s"%fname
    np.savez(fname,noiseadded_signal)
    print "xystarts are:"
    print xystarts
    print "size is:"
    print size
    print "Saving readout noise array as %s"%R_fname
    np.savez(R_fname,R_array)
    print "Saving the sky background array as %s"%B_sky_fname
    np.savez(B_sky_fname,B_sky_array)
    
    print "****** Done mock observation for distance %s ******"%distance
    print ""
    return data_FOV, xystarts, size, noiseadded_signal, B_sky_array, R_array

In [3]:
resolution = 14.; slicewidth = 20; exptime = 1000.*60.*60.; CMOS = False; ROnoise= None; x_center = 50.5; y_center = 13.5

distance = '500Mpc'
data_FOV_500, xystarts_500, size_500, noiseadded_signal_500, B_sky_array_500, R_array_500 = get_noisydata(resolution,distance,slicewidth,exptime,CMOS=CMOS,ROnoise=ROnoise,x_center=x_center,y_center=y_center)

distance = '200Mpc'
data_FOV_200, xystarts_200, size_200, noiseadded_signal_200, B_sky_array_200, R_array_200 = get_noisydata(resolution,distance,slicewidth,exptime,CMOS=CMOS,ROnoise=ROnoise,x_center=x_center,y_center=y_center)

distance = '100Mpc'
data_FOV_100, xystarts_100, size_100, noiseadded_signal_100, B_sky_array_100, R_array_100 = get_noisydata(resolution,distance,slicewidth,exptime,CMOS=CMOS,ROnoise=ROnoise,x_center=x_center,y_center=y_center)

distance = '50Mpc'
data_FOV_50, xystarts_50, size_50, noiseadded_signal_50, B_sky_array_50, R_array_50 = get_noisydata(resolution,distance,slicewidth,exptime,CMOS=CMOS,ROnoise=ROnoise,x_center=x_center,y_center=y_center)

****** Working on mock observation for distance 500Mpc ******
loading the data...
Loading data of slicewidth 20 (after rounding), number of slices is 4
data_500Mpc_14arcsec_20slwd.npz exists, loading now...
extracting the FOV...
adding noise to the data...
******* Adding noise to the input data to simulate a mock observation by the 48.0 lens Dragonfly Telescope *******
DEBUGGING: the binpix_size (resolution) is 14.0
DEBUGGING: the pixel size (inherent) is 2.8
VERBOSE: Using old cameras... (QE = 0.48, R = 10.)
DEBUGGING: R is : 10.0
DEBUGGING: the number of pixels per bin is 25.0
DEBUGGING: raise the data by 10** since was logged data before...
DEBUGGING: the total object signal [electrons] detected ranges from: 0.0172416 to 1.25239e+08
The width of the filter that the data was taken with is 3.0 nm. 
DEBUGGING: Sky background in filter width of 3.0 nm is calculated already: 1.473626.
Adding sky background noise and shot noise...
DEBUGGING: the background in the bandwidth is: 1.473626 ph

In [4]:
resolution = 14.; slicewidth = 20; exptime = 1000.*60.*60.; CMOS = False; ROnoise= None; x_center = 30.5; y_center = 67.5

distance = '500Mpc'
data_FOV_500, xystarts_500, size_500, noiseadded_signal_500, B_sky_array_500, R_array_500 = get_noisydata(resolution,distance,slicewidth,exptime,CMOS=CMOS,ROnoise=ROnoise,x_center=x_center,y_center=y_center)

distance = '200Mpc'
data_FOV_200, xystarts_200, size_200, noiseadded_signal_200, B_sky_array_200, R_array_200 = get_noisydata(resolution,distance,slicewidth,exptime,CMOS=CMOS,ROnoise=ROnoise,x_center=x_center,y_center=y_center)

distance = '100Mpc'
data_FOV_100, xystarts_100, size_100, noiseadded_signal_100, B_sky_array_100, R_array_100 = get_noisydata(resolution,distance,slicewidth,exptime,CMOS=CMOS,ROnoise=ROnoise,x_center=x_center,y_center=y_center)

distance = '50Mpc'
data_FOV_50, xystarts_50, size_50, noiseadded_signal_50, B_sky_array_50, R_array_50 = get_noisydata(resolution,distance,slicewidth,exptime,CMOS=CMOS,ROnoise=ROnoise,x_center=x_center,y_center=y_center)

****** Working on mock observation for distance 500Mpc ******
loading the data...
Loading data of slicewidth 20 (after rounding), number of slices is 4
data_500Mpc_14arcsec_20slwd.npz exists, loading now...
extracting the FOV...
adding noise to the data...
******* Adding noise to the input data to simulate a mock observation by the 48.0 lens Dragonfly Telescope *******
DEBUGGING: the binpix_size (resolution) is 14.0
DEBUGGING: the pixel size (inherent) is 2.8
VERBOSE: Using old cameras... (QE = 0.48, R = 10.)
DEBUGGING: R is : 10.0
DEBUGGING: the number of pixels per bin is 25.0
DEBUGGING: raise the data by 10** since was logged data before...
DEBUGGING: the total object signal [electrons] detected ranges from: 0.0136454 to 1.15728e+08
The width of the filter that the data was taken with is 3.0 nm. 
DEBUGGING: Sky background in filter width of 3.0 nm is calculated already: 1.473626.
Adding sky background noise and shot noise...
DEBUGGING: the background in the bandwidth is: 1.473626 ph

In [14]:
resolution = 14.; slicewidth = 5; exptime = 1000.*60.*60.; CMOS = False; ROnoise= None; x_center = 50.5; y_center = 13.5

distance = '500Mpc'
data_FOV_500, xystarts_500, size_500, noiseadded_signal_500, B_sky_array_500, R_array_500 = get_noisydata(resolution,distance,slicewidth,exptime,CMOS=CMOS,ROnoise=ROnoise,x_center=x_center,y_center=y_center)

distance = '200Mpc'
data_FOV_200, xystarts_200, size_200, noiseadded_signal_200, B_sky_array_200, R_array_200 = get_noisydata(resolution,distance,slicewidth,exptime,CMOS=CMOS,ROnoise=ROnoise,x_center=x_center,y_center=y_center)

distance = '100Mpc'
data_FOV_100, xystarts_100, size_100, noiseadded_signal_100, B_sky_array_100, R_array_100 = get_noisydata(resolution,distance,slicewidth,exptime,CMOS=CMOS,ROnoise=ROnoise,x_center=x_center,y_center=y_center)

distance = '50Mpc'
data_FOV_50, xystarts_50, size_50, noiseadded_signal_50, B_sky_array_50, R_array_50 = get_noisydata(resolution,distance,slicewidth,exptime,CMOS=CMOS,ROnoise=ROnoise,x_center=x_center,y_center=y_center)

****** Working on mock observation for distance 500Mpc ******
loading the data...
data_500Mpc_14arcsec.npz exists, loading now...
extracting the FOV...
adding noise to the data...
******* Adding noise to the input data to simulate a mock observation by the 48.0 lens Dragonfly Telescope *******
DEBUGGING: the binpix_size (resolution) is 14.0
DEBUGGING: the pixel size (inherent) is 2.8
VERBOSE: Using old cameras... (QE = 0.48, R = 10.)
DEBUGGING: R is : 10.0
DEBUGGING: the number of pixels per bin is 25.0
DEBUGGING: raise the data by 10** since was logged data before...
DEBUGGING: the total object signal [electrons] detected ranges from: 5.80868e-05 to 1.05697e+08
The width of the filter that the data was taken with is 1.0 nm. 
DEBUGGING: Sky background in filter width of 1.0 nm is calculated already: 0.560633.
Adding sky background noise and shot noise...
DEBUGGING: the background in the bandwidth is: 0.560633 photon/s/arcsec^2/m^2
DEBUGGING: the background signal, B_sky, is: 1.38247751

In [15]:
resolution = 14.; slicewidth = 5; exptime = 1000.*60.*60.; CMOS = False; ROnoise= None; x_center = 30.5; y_center = 67.5

distance = '500Mpc'
data_FOV_500, xystarts_500, size_500, noiseadded_signal_500, B_sky_array_500, R_array_500 = get_noisydata(resolution,distance,slicewidth,exptime,CMOS=CMOS,ROnoise=ROnoise,x_center=x_center,y_center=y_center)

distance = '200Mpc'
data_FOV_200, xystarts_200, size_200, noiseadded_signal_200, B_sky_array_200, R_array_200 = get_noisydata(resolution,distance,slicewidth,exptime,CMOS=CMOS,ROnoise=ROnoise,x_center=x_center,y_center=y_center)

distance = '100Mpc'
data_FOV_100, xystarts_100, size_100, noiseadded_signal_100, B_sky_array_100, R_array_100 = get_noisydata(resolution,distance,slicewidth,exptime,CMOS=CMOS,ROnoise=ROnoise,x_center=x_center,y_center=y_center)

distance = '50Mpc'
data_FOV_50, xystarts_50, size_50, noiseadded_signal_50, B_sky_array_50, R_array_50 = get_noisydata(resolution,distance,slicewidth,exptime,CMOS=CMOS,ROnoise=ROnoise,x_center=x_center,y_center=y_center)

****** Working on mock observation for distance 500Mpc ******
loading the data...
data_500Mpc_14arcsec.npz exists, loading now...
extracting the FOV...
adding noise to the data...
******* Adding noise to the input data to simulate a mock observation by the 48.0 lens Dragonfly Telescope *******
DEBUGGING: the binpix_size (resolution) is 14.0
DEBUGGING: the pixel size (inherent) is 2.8
VERBOSE: Using old cameras... (QE = 0.48, R = 10.)
DEBUGGING: R is : 10.0
DEBUGGING: the number of pixels per bin is 25.0
DEBUGGING: raise the data by 10** since was logged data before...
DEBUGGING: the total object signal [electrons] detected ranges from: 0.000181274 to 1.1174e+08
The width of the filter that the data was taken with is 1.0 nm. 
DEBUGGING: Sky background in filter width of 1.0 nm is calculated already: 0.560633.
Adding sky background noise and shot noise...
DEBUGGING: the background in the bandwidth is: 0.560633 photon/s/arcsec^2/m^2
DEBUGGING: the background signal, B_sky, is: 1.382477516

In [ ]:
#fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(16, 16))
#get_halpha_SB.makemap(noiseadded_signal_500,size_500,ax1,xystarts = xystarts_500)
#get_halpha_SB.makemap(noiseadded_signal_200,size_200,ax2,xystarts = xystarts_200)
#plt.tight_layout()